In [1]:
import numpy as np
import pandas as pd
import math
from scipy.optimize import fsolve

# 測試的

In [2]:
def f(x):
    return x + 1000
 
def g(y):
    return y * 3

def bestweight_pre(w):
    w0 = w[0]
    w1 = w[1]
    return [
        f(w0) + 3*g(w0) - 1,
        f(w1) + 3*g(w1) - 1
    ]

r = fsolve(bestweight_pre, [0,0])
print(r)

[-99.9 -99.9]


C:\Users\R\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


# 開發中的

In [3]:
def f1(x,arr_returns):
    arr_returns - rfrate 
    return sum( np.exp(-1*arr_returns/x) ) - len(arr_returns)
def get_riskiness_r(arr_returns):
    guess  = 10**(-5)
    while (guess<1000):
        risk2 = fsolve(f1,guess,arr_returns)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]

In [55]:
def risk(x):
    guess  = 10**(-5)
    while (guess<1000):
        risk2 = fsolve(f1,guess,x)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]
    #return np.std(x)

def finalrisk(weight,all_return):
    w = weight
    a = all_return
    portreturn = np.dot(np.array(all_return),np.array(weight).T)
    return risk(portreturn)

def sumrisk(weight,all_return):
    w = weight
    a = all_return
    v = 0
    for i in range(len(w)):
        v = v + w[i]*risk(w[i]*(all_return.iloc[:,i]))
    return v

def portretmean(weight,all_return):
    w = weight
    a = all_return
    return np.mean(np.dot(np.array(all_return),np.array(weight)))

# 這是原本峻安要的輸出的內容，是以4個weight值為例
def bestweight_sample(w):
    return [
        risk(w[0]*all_return.iloc[:,0]) - np.mean(all_return.iloc[:,0])*sumrisk(w,all_return) - np.mean(all_return.iloc[:,0]*w[0]) , 
        risk(w[1]*all_return.iloc[:,1]) - np.mean(all_return.iloc[:,1])*sumrisk(w,all_return) - np.mean(all_return.iloc[:,1]*w[1]) , 
        risk(w[2]*all_return.iloc[:,2]) - np.mean(all_return.iloc[:,2])*sumrisk(w,all_return) - np.mean(all_return.iloc[:,2]*w[2]) , 
        risk(w[3]*all_return.iloc[:,3]) - np.mean(all_return.iloc[:,3])*sumrisk(w,all_return) - np.mean(all_return.iloc[:,3]*w[3])
    ]



# 比較好的寫法，沒有限制weight的數量(亦即ETF的支數)
def bestweight(w):
    r = []
    restrict = sum(w) - 1
    r.append(restrict)
    for i in range(len(w)-1):
        v =(risk(w[i]*all_return.iloc[:,i]) \
            - (np.mean(all_return.iloc[:,i])-rfrate)*sumrisk(w,all_return) \
            - (np.mean(all_return.iloc[:,i]*w[i])-rfrate) )**4 \
            + (risk(w[i+1]*all_return.iloc[:,i+1]) \
            - (np.mean(all_return.iloc[:,i-1])-rfrate)*sumrisk(w,all_return) \
            - (np.mean(all_return.iloc[:,i+1]*w[i+1])-rfrate) )**4
        r.append(v) 
    return r

In [5]:
d1 = [-0.1,0.2,0.3,0.4]
d2 = [0.1,-0.2,-0.3,0.4]
d3 = [-0.1,0.3,0.4,0.5]
d4 = [-0.2,-0.4,0.5,0.6]
d5 = [-0.2,-0.4,0.6,0.8]
# d1-d5共有五個資產

rfrate = 0.2 

# 這邊應該是要再多一個轉置，意即pd.DataFrame([d1,d2,d3,d4,d5]).T
all_return = pd.DataFrame([d1,d2,d3,d4,d5]).T
print(all_return.shape)

# weight應該有五位數
weight = [0.1,0.2,0.2,0.5,-0.1]

#r1 = fsolve(bestweight_sample, weight)
#print(r1)

r2 = fsolve(bestweight, weight)
print(r2)
all_return

(4, 5)


C:\Users\R\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\R\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


[ 0.95666164 -0.47393711  0.22511148  0.39228776 -0.10012371]


,0,1,2,3,4
0,-0.1,0.1,-0.1,-0.2,-0.2
1,0.2,-0.2,0.3,-0.4,-0.4
2,0.3,-0.3,0.4,0.5,0.6
3,0.4,0.4,0.5,0.6,0.8


In [6]:
d1 = [0.049,0.05,0.05,0.06]
d2 = [0.028,0.03,0.07,0.04]
all_return = pd.DataFrame([d1,d2]).T
print(all_return.shape)


#weight = [0.1,0.2,0.2,0.5]
weight = [0.4,0.5]
all_return
r2 = fsolve(bestweight, weight)
print(r2)

(4, 2)


C:\Users\R\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


[0.34992637 0.64797366]


# 參考文章:
* [另外的python求解套件說明](https://github.com/wurmen/Gurobi-Python/blob/master/python-gurobi%20%20model/Python%2BGurobi%E5%BB%BA%E6%A8%A1.md)

# 比較sharpe ratio是否符合原本假設
## case1:正確解答

In [34]:
import numpy as np
x = np.array([-0.05,0.05,0.09,-0.01,0.04,0.01])
y = np.array([0.1,-0.01,0.01,-0.02,0.02,-0.05])
rfrate = 0.002

In [35]:
from scipy import optimize
def f(a):
    return np.std(a*x+(1-a)*y)/(np.sum(a*x+(1-a)*y)-rfrate) #倒數最小，原本值最大
abc = optimize.fmin(f, 0.7)
abc[0]

Optimization terminated successfully.
         Current function value: 0.267391
         Iterations: 12
         Function evaluations: 24


0.6073730468749996

## case2

<img src="picture/bestweight.PNG">


In [60]:
def penalty(w):
    if min(w)<0.05 or max(w)>0.95:
        return 10
    else:
        return 0
def bestweight(w):
    r = []
    for i in range(len(w)):
        v = risk(w[i]*(all_return.iloc[:,i]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i])-rfrate)/(portretmean(w,all_return)-rfrate) \
            + penalty(w)
        r.append(v) 
    return r

In [37]:
def risk(x):
    return np.std(x)

In [62]:
all_return = pd.DataFrame([x,y]).T
print(all_return.shape)
weight = [0.5,0.5]

r2 = fsolve(bestweight, weight)
print(r2)


(6, 2)


C:\Users\R\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\R\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


[0.44719264 0.55280736]


In [39]:
i = 0
w = [0.6,0.4]
risk(w[i]*(all_return.iloc[:,i]))/ sumrisk(w,all_return)  \
- (np.mean(all_return.iloc[:,i])-rfrate)/(portretmean(w,all_return)-rfrate) 

-0.23246829204480313

In [40]:
def ttttest(w):
    v = 0
    for i in range(len(w)):
        v = v + ( risk(w[i]*(all_return.iloc[:,i]))/ sumrisk(w,all_return)  \
        - (np.mean(all_return.iloc[:,i])-rfrate)/(portretmean(w,all_return)-rfrate) )**2
    return v

In [56]:

for i in range(50,65):
    a = i/100
    print(a,ttttest([a,1-a])) 

C:\Users\R\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


0.5 1.8353772701679452
0.51 1.7689800757234069
0.52 1.7025823310788335
0.53 1.636126996405491


C:\Users\R\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


0.54 1.6243357734784334
0.55 3.2414695584358997
0.56 4.237371695829466
0.57 4.710836124647462
0.58 5.042258515135968
0.59 5.327296997784045
0.6 5.599553404176629
0.61 5.8744806242010394
0.62 6.160863289672869
0.63 6.4647034788735676
0.64 6.790800395309272
0.5 0.018803411214407557
0.51 0.018298391899376543
0.52 0.017813500089931683
0.53 0.01734853354640198
0.54 0.016903455900468656
0.55 0.016478417004910983
0.56 0.016073779112456194
0.57 0.015690150227917057
0.58 0.015328426216008866
0.59 0.014989843427233083
0.6 0.014676043605643653
0.61 0.014389152435468953
0.62 0.014131871860409208
0.63 0.013907583595367944
0.64 0.013720456042369334


In [53]:
z1_rf = np.mean(x)-rfrate
z2_rf = np.mean(y)-rfrate
[z1_rf,z2_rf]
w = 0.6
rfrate = 0.002
portfo = w*x+(1-w)*y


In [54]:
risk(w*x) / (w*risk(w*x)+(1-w)*risk((1-w)*y)) - z1_rf/ (np.mean(portfo)-rfrate)

-0.23246829204480313